In [ ]:
import os
from glob import glob
import itertools

import numpy as np
import pandas as pd
import xarray as xr
import scipy.stats

%matplotlib widget
import matplotlib.pyplot as plt

import intake

In [ ]:
db = intake.open_catalog('data/MicroCTData/catalog.yml')

In [ ]:
df = db.chords.read()
ice_vol = db.ice_vol_frac.read()

metadata = db.information.read()
metadata

In [ ]:
def moment(cdl, order, material):
    return np.sum(cdl['i'] ** order * cdl['l_' + material]) / np.sum(cdl['l_' + material])

def moment_total(cdl, order):
    l_ice = cdl['l_ice'] / np.sum(cdl['l_ice'])
    l_pore = cdl['l_pore'] / np.sum(cdl['l_pore'])

    c_i_p = np.convolve(l_ice, l_pore)[0:len(l_ice)]

    g = l_ice + l_pore - 2 * c_i_p[0:len(l_ice)] + np.convolve(l_ice, c_i_p)[0:len(l_ice)] + np.convolve(l_pore, c_i_p)[0:len(l_ice)]
    return np.sum(cdl['i'] ** order * g) / np.sum(g)


stats = {f'{m}_mu{o}': df.groupby('site').apply(moment, order=o, material=m) \
                      for o in [0, 1, 2, 3, 4] for m in ['ice', 'pore']}

stats.update({f'mu{o}': df.groupby('site').apply(moment_total, order=o) \
                      for o in [0, 1, 2, 3, 4]})

stats = pd.DataFrame(stats).join(ice_vol.set_index('site'))

stats = stats.join(metadata.set_index('filename'))

In [ ]:
# impact of the different order in phi

Klp = (stats.ice_mu4 / (24*stats.ice_mu1**4))**(1/3)

phi_1 = 1 - stats.ice_vol_frac
Klp_semidilue = ((stats.ice_mu4/(24*stats.ice_mu1**4) - \
                 stats.ice_mu2*stats.ice_mu3/(6*stats.ice_mu1**5) * (1-phi_1) \
                 + stats.ice_mu2**3/(8*stats.ice_mu1**6) * (1-phi_1)**2 ) / phi_1**2) ** (1/3)

#phi_1 = 0.5
Klp_semidilue_terms = pd.DataFrame({
    'phi': stats.ice_vol_frac,
    't1': stats.ice_mu4/(24*stats.ice_mu1**4),
    't2': - stats.ice_mu2*stats.ice_mu3/(6*stats.ice_mu1**5) * (1-phi_1),
    't3':  stats.ice_mu2**3/(8*stats.ice_mu1**6)* (1-phi_1)**2})

In [ ]:
snowtypes = [['RG'], ['FC'], ['MF'], ['RG', 'FC', 'MF'], ['DH']]

for snowtype in snowtypes:
    mask = stats.main_type.isin(snowtype)
    print("----", snowtype)
    print("n=", mask.sum())
    n = mask.sum()
    mean = Klp_semidilue[mask].mean()
    std = Klp_semidilue[mask].std()
    print(f"{mean:.2} +- {std:.2} (n={n})")
    print(Klp_semidilue[mask].min(), Klp_semidilue[mask].max())
    
for snowtype1, snowtype2 in itertools.product(snowtypes, snowtypes):
    if snowtype1 is snowtype2:
        continue

    mask1 = stats.main_type.isin(snowtype1)
    mask2 = stats.main_type.isin(snowtype2)
    
    t = scipy.stats.ttest_ind(Klp_semidilue[mask1], Klp_semidilue[mask2], equal_var = False)

    print(snowtype1, snowtype2, "p=", t.pvalue)

In [ ]:
stats['K'] = Klp_semidilue  #[normalsnow].sort_values().to_csv('output-polydispersity.csv')
stats.to_csv('results/polydispersity-all-data.csv')

In [ ]:
normalsnow = stats.main_type.isin(['RG', 'FC', 'MF'])
DHsnow = stats.main_type.isin(['DH'])

blue = "#1f77b4"
orange = "#ff7f0e"

f, axs = plt.subplots(1, 1, figsize=(8.7, 3.4), sharey=True)

Kl_special = {'Antarctic snow\n(from\nmicrowaves)': (0.63, '--'),
              #'Arctic Depth Hoar\n(from microwaves)': (1.48, '-'),
              'Sparse\nspheres\n(from theory)': ((9/128)**(1/3), ':'),
              'Debye random medium\n(from theory)': (1, ':'),
             }
for label, (K, s) in Kl_special.items():
    #axs.plot([K, K],[0, 24], s, label=label)
    line = 0 if 'Depth Hoar' in label else 1.05
    
    axs.arrow(K, 0, 0, 35, edgecolor=blue, linestyle=s, linewidth=2)
    axs.annotate(label, xy=[K, line], 
                 xycoords=('data', 'axes fraction'),
                 horizontalalignment='center'
                )
        
axs.arrow(0.8, -20, 0.4, 0, edgecolor=orange, linewidth=2)
axs.annotate("Finnish depth hoar\n(from $\mu$-CT, Leinss et al. 2020)", xy=(1, -20-8), horizontalalignment='center')

axs.arrow(1.2, -15, 0.7, 0, edgecolor=orange, linestyle='--', linewidth=2)
axs.annotate("Canadian arctic depth hoar\n(from microwaves)", xy=(1.5, -17-7), horizontalalignment='center')

bins = np.arange(0.4, 1.5, 0.04)
#axs[0].hist(Kl[normalsnow], alpha=0.3, bins=bins, label='Rounded and faceted grains (from $\mu$CT)')
#axs[0].hist(Kl[DHsnow], alpha=0.3, bins=bins, label='Alpine Depth hoar (from $\mu$CT)')

axs.hist(Klp_semidilue[normalsnow], alpha=0.5, bins=bins, edgecolor='0.5', label='Alpine convex grains\n(from CLD)')
#axs.hist(Klp_semidilue[DHsnow], hatch='//\\\\', edgecolor='0.6', alpha=0.3, bins=bins, histtype='step', label='Alpine Depth Hoar (from chord length distribution)')
h, e = np.histogram(Klp_semidilue[DHsnow], bins=bins)
axs.bar(e[:-1], -h, width=np.diff(e), 
        align='edge', edgecolor='0.6', facecolor='#ff7f0e', alpha=0.3, label='Alpine Depth Hoar\n(from CLD)')


#axs.set_frame(False)
axs.set_ylim((-30, 30))
axs.set_xlabel("Polydispersity $K$", loc='right', labelpad=-33)
axs.set_ylabel("Number of samples")
#axs.yaxis.visible = True
axs.spines['bottom'].set_position('zero')

axs.spines['left'].set_visible(True)
axs.spines['top'].set_visible(False)
axs.spines['right'].set_visible(False)
#axs.yaxis.set_tick_params(left='off', right='off', length=0, labelsize=0)
axs.yaxis.set_tick_params(right='off')
axs.yaxis.set_major_formatter(lambda x, pos: int(abs(x)))

axs.legend(framealpha=0.3, prop={'size': 8}, loc='lower left')
f.tight_layout()
f.savefig("fig-polydispersity.pdf")


